# M365 Incident Response — Compromised Email (App-Only)

This notebook is a starter playbook for automated incident response (IR) focused on a potentially compromised Microsoft 365 mailbox. It is written for the .NET PowerShell kernel (PowerShell via .NET Interactive) and uses app-only (client credentials) authentication against Microsoft Graph so it can be run unattended by a service account or automation pipeline.

Security note: do NOT commit client secrets or certificates into source control. Store secrets in a secure location (Key Vault, files with strict permissions, or environment variables) and restrict access to the notebook and host environment.

## Setup and Authentication

Configure app-only authentication to Microsoft Graph.

In [ ]:
# Install Microsoft Graph PowerShell SDK if not already installed
# Install-Module Microsoft.Graph -Scope CurrentUser -Force

# Configure app registration details
$tenantId = "YOUR_TENANT_ID"
$clientId = "YOUR_CLIENT_ID"
$clientSecret = "YOUR_CLIENT_SECRET"  # Store securely!

# Convert client secret to secure string
$secureClientSecret = ConvertTo-SecureString -String $clientSecret -AsPlainText -Force
$clientSecretCredential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $clientId, $secureClientSecret

# Connect to Microsoft Graph
Connect-MgGraph -TenantId $tenantId -ClientSecretCredential $clientSecretCredential

Write-Host "Successfully connected to Microsoft Graph"

## Configuration

Set the compromised user's email address and incident parameters.

In [ ]:
# Compromised user configuration
$compromisedUserEmail = "user@contoso.com"
$investigationStartDate = (Get-Date).AddDays(-7)  # Look back 7 days

Write-Host "Investigating user: $compromisedUserEmail"
Write-Host "Investigation period: From $investigationStartDate to now"

## Step 1: Gather User Information

Retrieve basic information about the compromised user.

In [ ]:
# Get user details
try {
    $user = Get-MgUser -UserId $compromisedUserEmail -Property DisplayName,UserPrincipalName,AccountEnabled,SignInActivity,Id -ErrorAction Stop
    
    $user | Select-Object DisplayName, UserPrincipalName, AccountEnabled, Id
    
    # Store user ID for later use
    $userId = $user.Id
    
    if (-not $userId) {
        throw "User ID could not be retrieved"
    }
    
    Write-Host "User found: $($user.DisplayName)" -ForegroundColor Green
} catch {
    Write-Host "ERROR: Failed to retrieve user: $_" -ForegroundColor Red
    Write-Host "Please verify the user email address and try again." -ForegroundColor Yellow
    throw
}

## Step 2: Review Recent Sign-In Activity

Check for suspicious sign-in attempts and locations.

In [ ]:
# Get sign-in logs for the user
$signInLogs = Get-MgAuditLogSignIn -Filter "userPrincipalName eq '$compromisedUserEmail' and createdDateTime ge $($investigationStartDate.ToString('yyyy-MM-ddTHH:mm:ssZ'))" -Top 100

# Display sign-in summary
$signInLogs | Select-Object CreatedDateTime, AppDisplayName, IpAddress, Location, Status | Format-Table -AutoSize

# Identify unique IP addresses
$uniqueIPs = $signInLogs | Select-Object -ExpandProperty IpAddress -Unique
Write-Host "Unique IP addresses: $($uniqueIPs -join ', ')"

## Step 3: Check Mailbox Audit Logs

Review mailbox operations and forwarding rules.

In [ ]:
# Connect to Exchange Online if needed
# Connect-ExchangeOnline -CertificateThumbprint "YOUR_CERT_THUMBPRINT" -AppId $clientId -Organization "contoso.onmicrosoft.com"

# Search mailbox audit logs
$startDate = $investigationStartDate
$endDate = Get-Date

# Example: Search for specific operations
# $auditLogs = Search-UnifiedAuditLog -StartDate $startDate -EndDate $endDate -UserIds $compromisedUserEmail -Operations "New-InboxRule","Set-InboxRule","UpdateInboxRules" -ResultSize 1000

Write-Host "Audit log search configured for date range: $startDate to $endDate"

## Step 4: Examine Inbox Rules

Check for suspicious forwarding or deletion rules.

In [ ]:
# Get inbox rules (requires Exchange Online connection)
# $inboxRules = Get-InboxRule -Mailbox $compromisedUserEmail

# Display rules that forward or delete messages
# $suspiciousRules = $inboxRules | Where-Object { $_.ForwardTo -or $_.ForwardAsAttachmentTo -or $_.DeleteMessage -eq $true }

# if ($suspiciousRules) {
#     Write-Host "WARNING: Found potentially suspicious inbox rules:" -ForegroundColor Red
#     $suspiciousRules | Format-List Name, Description, ForwardTo, ForwardAsAttachmentTo, DeleteMessage
# } else {
#     Write-Host "No suspicious inbox rules found." -ForegroundColor Green
# }

Write-Host "Inbox rule check configured (requires Exchange Online connection)"

## Step 5: Review Mailbox Permissions

Check for unauthorized delegate access.

In [ ]:
# Get mailbox permissions (requires Exchange Online)
# $mailboxPermissions = Get-MailboxPermission -Identity $compromisedUserEmail | Where-Object { $_.User -ne "NT AUTHORITY\SELF" }

# if ($mailboxPermissions) {
#     Write-Host "Mailbox permissions found:"
#     $mailboxPermissions | Format-Table User, AccessRights, IsInherited
# } else {
#     Write-Host "No additional mailbox permissions found."
# }

Write-Host "Mailbox permission check configured"

## Step 6: Check OAuth App Consents

Identify potentially malicious OAuth applications with mail access.

In [ ]:
# Get OAuth grants for the user
$oauthGrants = Get-MgUserOauth2PermissionGrant -UserId $userId

# Display apps with mail-related permissions
foreach ($grant in $oauthGrants) {
    $app = Get-MgServicePrincipal -ServicePrincipalId $grant.ClientId
    
    if ($grant.Scope -match "Mail") {
        Write-Host "App: $($app.DisplayName)" -ForegroundColor Yellow
        Write-Host "  Permissions: $($grant.Scope)"
        Write-Host "  Consent Type: $($grant.ConsentType)"
    }
}

Write-Host "OAuth app consent review completed"

## Step 7: Review Sent Items

Check for suspicious sent messages.

In [ ]:
# Get recent sent messages
$sentMessages = Get-MgUserMessage -UserId $userId -Filter "sentDateTime ge $($investigationStartDate.ToString('yyyy-MM-ddTHH:mm:ssZ')) and isDraft eq false" -Top 50 -OrderBy "sentDateTime DESC"

# Display sent message summary
$sentMessages | Select-Object SentDateTime, Subject, @{Name="To";Expression={$_.ToRecipients.EmailAddress.Address -join ", "}} | Format-Table -AutoSize

Write-Host "Total sent messages in investigation period: $($sentMessages.Count)"

## Step 8: Immediate Remediation Actions

Take containment actions to limit further damage.

In [ ]:
# CAUTION: Uncomment and run these commands carefully

# Option 1: Revoke all refresh tokens (forces re-authentication)
# Revoke-MgUserSignInSession -UserId $userId
# Write-Host "All user sessions revoked" -ForegroundColor Green

# Option 2: Disable the account temporarily
# Update-MgUser -UserId $userId -AccountEnabled:$false
# Write-Host "Account disabled" -ForegroundColor Red

# Option 3: Remove suspicious inbox rules (requires Exchange Online)
# foreach ($rule in $suspiciousRules) {
#     Remove-InboxRule -Identity $rule.Identity -Mailbox $compromisedUserEmail -Confirm:$false
#     Write-Host "Removed rule: $($rule.Name)" -ForegroundColor Yellow
# }

Write-Host "Remediation actions configured (review and uncomment to execute)"

## Step 9: Reset Password

Force a password reset for the compromised account.

In [ ]:
# Generate temporary password
# $tempPassword = "TempP@ss" + (Get-Random -Minimum 1000 -Maximum 9999) + "!"
# $passwordProfile = @{
#     Password = $tempPassword
#     ForceChangePasswordNextSignIn = $true
# }

# Reset password
# Update-MgUser -UserId $userId -PasswordProfile $passwordProfile
# Write-Host "Password has been reset. Temporary password: $tempPassword" -ForegroundColor Yellow
# Write-Host "User will be required to change password at next sign-in" -ForegroundColor Green

Write-Host "Password reset configured (uncomment to execute)"

## Step 10: Enable MFA

Ensure multi-factor authentication is enabled.

In [ ]:
# Check current MFA status
$authMethods = Get-MgUserAuthenticationMethod -UserId $userId

Write-Host "Current authentication methods:"
$authMethods | Select-Object Id, @{Name="Type";Expression={$_.AdditionalProperties['@odata.type']}} | Format-Table

# Enable MFA registration (requires appropriate policies)
Write-Host "Ensure MFA is enforced via Conditional Access policies" -ForegroundColor Yellow

## Step 11: Generate Incident Report

Summarize findings and actions taken.

In [ ]:
# Create incident summary
$report = @"
===================================
M365 INCIDENT RESPONSE REPORT
===================================
User: $compromisedUserEmail
Investigation Date: $(Get-Date -Format 'yyyy-MM-dd HH:mm:ss')
Investigation Period: $investigationStartDate to $(Get-Date)

FINDINGS:
- Sign-in attempts: $($signInLogs.Count)
- Unique IP addresses: $($uniqueIPs.Count)
- Sent messages: $($sentMessages.Count)
- OAuth grants with mail access: Review output above

ACTIONS TAKEN:
- Investigation completed
- Review remediation section for containment actions

RECOMMENDATIONS:
- Review and remove suspicious inbox rules
- Revoke refresh tokens to force re-authentication
- Reset user password
- Enable MFA if not already active
- Review and revoke suspicious OAuth app consents
- Monitor user activity for next 30 days
===================================
"@

Write-Host $report

# Optionally save to file
# $report | Out-File -FilePath "incident-report-$(Get-Date -Format 'yyyyMMdd-HHmmss').txt"

## Cleanup

Disconnect from Microsoft Graph.

In [ ]:
# Disconnect from Microsoft Graph
Disconnect-MgGraph

Write-Host "Disconnected from Microsoft Graph" -ForegroundColor Green